<p style="text-align: center"> Escuela Técnica Superior de Ingeniería Informática</p>
<p style="text-align: center">Universidad de Sevilla</p>
<p style="text-align: center">Máster en Lógica, Ciencias de la Computación e Inteligencia Artificial </p>
<p style="text-align: center">Procesamiento del Lenguaje Natural </p>
<p style="text-align: center">Evaluación Continua: Identificación del lenguaje y limpieza de datasets </p>
<p> </p>


In [1]:
import numpy as np
from pathlib import Path
import random
import itertools
import re
import nltk
from nltk import FreqDist
import pandas as pd
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords

# Preparación del dataset de trabajo

## Tarea 1: Construcción de ficheros con los corpus

La primera tarea que se be realizar una vez descargados los ficheros es construir los 
datasets de entrenamiento y evaluación, y además hacerlo de una forma balanceada.
Para ello, se han de generar ficheros de entrenamiento para cada uno de los lenguajes 
elegidos formados por 100.000 líneas, y un fichero de evaluación formado por 10.000 
líneas. Evidentemente el contenido del fichero de evaluación debe ser diferente al de 
entrenamiento. Una estrategia adecuada es utilizar un modelo de selección aleatoria de 
forma que al recorrer el fichero original, de cada 11 líneas recorridas, 10 (seleccionadas 
aleatoriamente) sean enviadas al fichero de entrenamiento y 1 al de evaluación.

Para poder acceder a los directorios de forma muy fácil, se va a emplear la librería ```Pathlib```.

In [2]:
# URL que contiene todos los elementos (MODIFICAR CON EL DIRECTORIO DONDE ESTÉN LOS FICHEROS EN CRUDO)
directory = Path(r"C:\Users\PC\Documents\Master_MULCIA\2ºCuatrimestre\PLN\data")

for file in directory.glob("europarl-v7*"):
    with open(file, 'r',encoding='utf-8') as file_org, \
     open(f'{directory}/{file.name[-2:]}_train.txt', 'w',encoding='utf-8') as file_train, \
     open(f'{directory}/{file.name[-2:]}_test.txt', 'w',encoding='utf-8') as file_test:
        cnt_test,cnt_train=0,0
        for i, line in enumerate(file_org):
            if random.randint(1, 11) == 1 and cnt_test < 10000:
                file_test.write(line)
                cnt_test+=1
            elif random.randint(1, 11) != 1 and cnt_train <100000:
                file_train.write(line)
                cnt_train+=1
            if cnt_test+cnt_train == 110000:
                break

## Tarea 2: 

La segunda tarea de este apartado consiste en normalizar los corpus de evaluación y 
entrenamiento. Está claro que para esta tarea los signos de puntuación no aportan ninguna 
información. Por tanto, se deben normalizar los textos de forma que se eliminen los 
signos de puntuación, así como las expresiones numéricas. A la hora de realizar esta 
operación hay que tener cuidado con signos que sí pueden ser relevantes como puede ser 
el apóstrofe en los lenguajes donde se aplique. Por tanto, como parte de esta tarea se ha 
de indicar qué signos se eliminan, y cuáles no, justificando la decisión. Así mismo, podría 
ser interesante normalizar el texto en cuanto a mayúsculas. No obstante, este criterio se 
deja a decisión de cada alumno, debiendo justificar por qué toma una u otra decisión.


Como cada idioma tiene sus características y peculiaridades, por lo que la limpieza de los textos debe ser adaptada a cada idioma. Por ejemplo, en español y francés, es importante no borrar los acentos ortográficos, mientras que en inglés no tiene.

Así que se va a generar unas funciones para poder limpiar cada corpus de cada idioma.

En primer lugar, hemos generado una función para eliminar las características irrelevantes de todos los idiomas. Como los signos de puntuación, paréntesis, y caracteres numéricos. También, se ha decidido normalizar los textos en cuanto a las mayúsculas, esto es debido a que nos puede traer beneficios como reducir la dispersión de los datos, mejorar la consistencia y simplifica el preprocesado.

Para la justificación de lo que se ha borrado en los idiomas se ha empleado la [página de Babel](https://www.babbel.com/en/magazine/punctuation-marks)

In [3]:
def clean_txt(txt:str) -> str:
    txt = txt.lower()
    txt = txt.replace("/", "")
    txt = re.sub(r'[.,;:!?()\[\]{}—"\']|-]', '', txt)
    txt = re.sub(r'\d+', '', txt)
    txt = re.sub(r'\s+', ' ', txt)
    return txt

En español, eliminamos los signos de puntuación como la apertura de interrogación y exclamación, que son relevantes para el español,
pero no para el resto de lenguajes. Además, las comillas también se pueden quitar.

In [4]:
stop_words_es = set(stopwords.words('spanish'))
def clean_txt_es(txt:str) -> str:
    txt = clean_txt(txt)
    txt = re.sub(r'[¿¡]', '', txt)
    txt = txt.replace("«", "")
    txt = txt.replace("»", "")
    txt = re.sub(r"['\"]", "", txt)
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_es]
    txt_clean = ' '.join(words_clean)
    return txt_clean

En francés se pueden eliminar las mismas comillas que usamos los españoles, al igual que el italiano, debido a que vienen de la misma familia.

In [5]:
stop_words_fr = set(stopwords.words('french'))
def clean_txt_fr(txt):
    txt = clean_txt(txt)
    txt = txt.replace("«", "")
    txt = txt.replace("»", "")
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_fr]
    txt_clean = ' '.join(words_clean)
    return txt_clean

In [6]:
stop_words_it = set(stopwords.words('italian'))
def clean_txt_it(txt):
    txt = clean_txt(txt)
    txt = txt.replace("«", "")
    txt = txt.replace("»", "")
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_it]
    txt_clean = ' '.join(words_clean)
    return txt_clean

En inglés y alemán se parecen mucho, debido a que vienen de la misma familia, por lo que podemos eliminar las comillas.

In [7]:
stop_words_en = set(stopwords.words('english'))

def clean_txt_en(txt):
    txt = clean_txt(txt)
#     txt = re.sub(r'[„“]', '', txt)
    txt = re.sub(r"[‘’'“”\"”]+", "", txt)
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_en]
    txt_clean = ' '.join(words_clean)
    
    return txt_clean
    

In [8]:
stop_words_de = set(stopwords.words('german'))
def clean_txt_de(txt):
    txt = clean_txt(txt)
    txt = re.sub(r"[‘’'“”\"”]+", "", txt)
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_de]
    txt_clean = ' '.join(words_clean)
    return txt_clean

En polaco, NLTK [no tiene su propio conjunto de palabras stopwords](https://stackoverflow.com/questions/54573853/nltk-available-languages-for-stopwords). Por lo que se ha seleccionado un un fichero con [stopwords en polaco](https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt).

In [9]:
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [10]:
stop_words_pl = []
with open('polish.stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stop_words_pl.append(line.strip())
stop_words_pl = set(stop_words_pl)

In [11]:
def clean_txt_pl(txt):
    txt = clean_txt(txt)
    txt = txt.replace('-', '')
    words = txt.split()
    words_clean = [word for word in words if word.lower() not in stop_words_pl]
    txt_clean = ' '.join(words_clean)
    return txt_clean

Generamos un diccionario con las distintas funciones y no tener que usar condicionales

In [12]:
funciones_limpieza = {
    'es': clean_txt_es,
    'fr': clean_txt_fr,
    'it': clean_txt_it,
    'de': clean_txt_de,
    'en': clean_txt_en,
    'pl': clean_txt_pl,
}

In [13]:
for file in directory.glob('*_[train|test]*'):
    with open(file, 'r',encoding='utf-8') as file_org:
        language = file.name.split("_")[0]
        file_name = file.name.split(".")[0]
        clean_text = ""
        for line in file_org:
            clean_line = funciones_limpieza[language](line)
            clean_text += clean_line
        with open(f'{directory}/{file.name.split(".")[0]}_clean.txt', 'w',encoding='utf-8') as file_dest:
            file_dest.write(clean_text)
        

## Tarea 3: Obtener las  100 palabras más frecuentes del corpues de entrenamiento

La tercera tarea de este apartado consistirá en la obtención, para cada uno de los 6 
lenguajes elegidos, de las 100 palabras más frecuentes en el corpus de entrenamiento. 

In [14]:
dicc={}
for file in directory.glob('*_test_clean*'):
    with open(file, 'r', encoding='utf-8') as file_org:
        text=file_org.read()
        language = file.name.split("_")[0]
        words = nltk.tokenize.word_tokenize(text)
        fdist = FreqDist(words)
        top_words = [word for word, frequency in fdist.most_common(100)]
        dicc[language]=top_words

In [15]:
table = pd.DataFrame(dicc)
table

,de,en,es,fr,it,pl
0,kommission,european,comisión,a,commissione,panie
1,europäischen,commission,presidente,plus,presidente,przewodniczący
2,union,president,unión,commission,essere,komisji
3,präsident,also,europea,cette,europea,europejskiej
4,möchte,would,parlamento,président,stati,unii
...,...,...,...,...,...,...
95,staaten,system,programa,grande,sociale,została
96,schutz,given,momento,certains,necessario,jedynie
97,ziel,information,sector,partie,programma,panu
98,situation,much,asuntos,beaucoup,altro,naszej


# Implementación de un modelo TF-IDF para detección del lenguaje

#  Evaluación del modelo de detección de lenguajes mediante TF-IDF

# Limpieza de los corpus de entrenamiento y evaluación

# TF-IDF con el modelo de corpus limpio

# Detección de lenguajes usando n-gramas